# TUTORIAL: ¿CÓMO USAR GRANDES MODELOS DE LENGUAJE (LLMs) GRATUITOS DESDE HUGGING FACE? (PIPELINES Y LLMs)

En este tutorial veremos la forma más sencilla de usar LLMs, para diferentes tareas de procesamiento y generación de lenguaje natural, haciendo uso de los "pipelines" de Hugging Face.

1. ¿Cómo buscar un LLM en Hugging Face?
2. Breve repaso: tokens y LLMs
3. Clasificación de texto con pipelines
4. Tarea pregunta-respuesta con pipelines
5. Generación de resúmenes con pipelines
6. Generación de texto con pipelines

## 1. ¿Cómo buscar un LLM en Hugging Face?

A continuación los pasos a seguir:

1.  Identificar el tipo de tarea que queremos implementar (clasificación de texto, embeddings, generación de texto, etc.)
2. Buscar la familia de modelos asociada en Hugging Face
3. Filtrar por aquellos modelos pre-entrenados para el lenguaje en el que tenemos los datos
4. Ordenar modelos por más descargados y analizar la tarjeta del modelo para verificar ventana de contexto, tamaño en memoria, etc.
5. Copiar la ruta del modelo ("model_path")

## 2. Breve repaso: tokens y LLMs

![](https://drive.google.com/uc?export=view&id=1bKAfOHIIWIhXBE11Iy3gsEByZTL2miFe)

## 3. Clasificación de texto con pipelines

Usaremos el modelo "cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual".

Al definir un "pipeline" siempre debemos incluir al menos estos cuatro argumentos:

1. El tipo de tarea
2. La ruta del tokenizer y la ruta del modelo (¡ambas son la misma!)
3. Definir el dispositivo a usar (GPU o CPU)

Creemos un "pipeline" básico para clasificación de texto:

In [1]:
from transformers import pipeline

model_path = 'cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual'
pipe = pipeline(
    task = "text-classification",
    model=model_path,
    tokenizer=model_path,
    top_k=None, # *** retornar todas las probabilidades asociadas a las categorías
    device_map="auto"
)

config.json:   0%|          | 0.00/982 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cpu


Y una vez definida la "pipeline" podemos generar predicciones con el modelo.

Por ejemplo, usemos el siguiente *review* de una película para determinar si el sentimiento es positivo, negativo o neutral:

In [2]:
# Review
review = """
Recuerdo ver esta película en el cine y salir encantado con esta comedia musical. Después de varios
años he decidido verla nuevamente pero con el miedo de comprobar si con el paso de los años este
film ha perdido la calidad que recordaba.Pero para mi sorpresa debo decir que la película sigue
resultando interesante y divertida. Es cierto que no brilla con tanta luz como recordaba, pero
también debo decir que no flojea tanto como esperaba. Es una comedia musical que entretiene y con el
que uno pasa un rato muy agradable.El guion es divertido y funciona bien. Nos plantea a dos parejas
en el que un miembro de cada una de ellas empiezan a tener una relación paralela siendo infiel a su
novio y novia. Por consiguiente, la trama es una sucesión de escenas llenas de líos de infidelidad y
situaciones casi surrealistas; donde los personajes manejan las relaciones como si fuera un
espectáculo de malabarismos.
"""
print(review)


Recuerdo ver esta película en el cine y salir encantado con esta comedia musical. Después de varios
años he decidido verla nuevamente pero con el miedo de comprobar si con el paso de los años este
film ha perdido la calidad que recordaba.Pero para mi sorpresa debo decir que la película sigue
resultando interesante y divertida. Es cierto que no brilla con tanta luz como recordaba, pero
también debo decir que no flojea tanto como esperaba. Es una comedia musical que entretiene y con el
que uno pasa un rato muy agradable.El guion es divertido y funciona bien. Nos plantea a dos parejas
en el que un miembro de cada una de ellas empiezan a tener una relación paralela siendo infiel a su
novio y novia. Por consiguiente, la trama es una sucesión de escenas llenas de líos de infidelidad y
situaciones casi surrealistas; donde los personajes manejan las relaciones como si fuera un
espectáculo de malabarismos.



Para generar una predicción con la "pipeline" simplemente presentamos el texto como argumento de dicha "pipeline":

In [3]:
# Predicción
preds = pipe(review)
preds

[[{'label': 'positive', 'score': 0.7006554007530212},
  {'label': 'neutral', 'score': 0.22694571316242218},
  {'label': 'negative', 'score': 0.07239888608455658}]]

Y siempre será necesario **post-procesar** la salida para extraer el resultado que nos interesa:

In [4]:
# Determinar categoría (la probabilidad más alta) a partir de preds

# Tomar el primer (y único) elemento de la lista
preds = preds[0]

# Iterar
cat = max(preds, key=lambda x: x['score'])
cat = cat['label']
print(cat)


positive


Y todas las "pipelines" tienen varios parámetros que se pueden ajustar.

Por ejemplo, en lugar de retornar la distribución de probabilidad de todas las categorías ("top_k = None") retornemos sólo la categoría más probable ("top_k = 1"):

In [5]:
# Retornar top-1 (categoría más probable)
pipe = pipeline(
    task = "text-classification",
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device_map="auto",
    top_k=1
)
preds = pipe(review)
preds

Device set to use cpu


[[{'label': 'positive', 'score': 0.7006554007530212}]]

## 4. Tarea pregunta-respuesta con "pipelines"

La lógica es muy similar a la que vimos en el caso anterior. Lo único que cambia es que por ser una tarea diferente algunos argumentos serán diferentes.

Comencemos creando la "pipeline" con el modelo "dccuchile/distilbert-base-spanish-uncased-finetuned-qa-mlqa":

In [6]:
model_path = "dccuchile/distilbert-base-spanish-uncased-finetuned-qa-mlqa"
pipe = pipeline("question-answering",
                model=model_path,
                tokenizer=model_path,
                device_map="auto")

config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/267M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/267M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

Device set to use cpu


En este caso de pregunta-respuesta el modelo recibe un contexto ("context") y una pregunta ("question") y a la salida entrega un texto (proveniente del contexto) que contiene la posible respuesta a la pregunta.

Creemos el contexto y la pregunta:

In [7]:
contexto = """La inteligencia artificial es una rama de la informática
que busca crear sistemas capaces de realizar tareas que normalmente
requieren inteligencia humana, como el reconocimiento de voz,
la traducción automática y la visión por computadora."""

pregunta = "¿Qué busca crear la inteligencia artificial?"

Y ahora le presentamos esta información al modelo para que busque una respuesta:

In [8]:
pred = pipe(question=pregunta, context=contexto)
print(pred)

{'score': 0.30732041597366333, 'start': 73, 'end': 154, 'answer': 'sistemas capaces de realizar tareas que normalmente\nrequieren inteligencia humana'}


Acá:

- "score" indica qué tan confiado está el modelo de que la respuesta entregada es precisamente una respuesta a la pregunta
- "start", "end": posiciones en el texto de entrada que contienen la respuesta
- "answer": el texto con la respuesta. **No es un texto generado, pues estos modelos no son generativos**

Hagamos el post-procesamiento de la predicción para extraer únicamente el texto de la respuesta:

In [9]:
print(f"contexto: {contexto}")
print('-'*20)
print(f"pregunta: {pregunta}")
print('-'*20)
print(f"respuesta: {pred['answer']}")


contexto: La inteligencia artificial es una rama de la informática
que busca crear sistemas capaces de realizar tareas que normalmente
requieren inteligencia humana, como el reconocimiento de voz,
la traducción automática y la visión por computadora.
--------------------
pregunta: ¿Qué busca crear la inteligencia artificial?
--------------------
respuesta: sistemas capaces de realizar tareas que normalmente
requieren inteligencia humana


En este caso podemos usar el argumento "handle_impossible_answer = True" para que, en caso de que el modelo no encuentre una respuesta, retorne un *string* vacío:

In [10]:
# Manejar respuestas imposibles
pipe = pipeline("question-answering",
                model=model_path,
                tokenizer=model_path,
                device_map="auto",
                handle_impossible_answer=True)

contexto = """La inteligencia artificial es una rama de la informática
que busca crear sistemas capaces de realizar tareas que normalmente
requieren inteligencia humana, como el reconocimiento de voz,
la traducción automática y la visión por computadora."""

pregunta = "¿Cómo preparar una torta?"

pred = pipe(question=pregunta, context=contexto)
print(pred)

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

Device set to use cpu


{'score': 0.1922626793384552, 'start': 0, 'end': 0, 'answer': ''}


## 5. Generación de resúmenes con "pipelines"

En este caso se usan modelos texto-a-texto es decir modelos generativos que contienen tanto el codificador como el decodificador de una Red Transformer.

Para este ejemplo usaremos el modelo "csebuetnlp/mT5_multilingual_XLSum":

In [11]:
model_path = "csebuetnlp/mT5_multilingual_XLSum"
pipe = pipeline("summarization",
                model=model_path,
                tokenizer=model_path,
                device_map="auto")

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cpu


In [12]:
texto = """
Los videos que afirman que las vacunas aprobadas son peligrosas y causan autismo,
cáncer o infertilidad se encuentran entre los que serán eliminados,
según informó la compañía.

La política incluye la cancelación de las cuentas de influencers antivacunas.

Los gigantes tecnológicos han sido criticados por no hacer más para contrarrestar
la información falsa sobre salud en sus sitios web.

En julio, el presidente estadounidense Joe Biden afirmó que las plataformas de
redes sociales eran en gran medida responsables del escepticismo de las personas
a la hora de vacunarse, al difundir información errónea,
y les pidió que abordaran el problema.

YouTube, propiedad de Google, informó que se eliminaron 130.000 videos de su
plataforma desde el año pasado, cuando implementó una prohibición sobre el
contenido que difundía información errónea sobre las vacunas.

En una publicación de blog, la compañía afirmó haber visto cómo las afirmaciones
falsas sobre las vacunas contra la COVID-19 "se extendían a la desinformación
sobre las vacunas en general". La nueva política abarca vacunas aprobadas
desde hace tiempo, como las del sarampión o la hepatitis B.

«Estamos ampliando nuestras políticas contra la desinformación médica
en YouTube con nuevas directrices sobre las vacunas que se administran
actualmente, cuya seguridad y eficacia han sido confirmadas por las autoridades
sanitarias locales y la OMS», decía la publicación, en referencia a la
Organización Mundial de la Salud.
"""
print(texto)


Los videos que afirman que las vacunas aprobadas son peligrosas y causan autismo,
cáncer o infertilidad se encuentran entre los que serán eliminados,
según informó la compañía.

La política incluye la cancelación de las cuentas de influencers antivacunas.

Los gigantes tecnológicos han sido criticados por no hacer más para contrarrestar
la información falsa sobre salud en sus sitios web.

En julio, el presidente estadounidense Joe Biden afirmó que las plataformas de
redes sociales eran en gran medida responsables del escepticismo de las personas
a la hora de vacunarse, al difundir información errónea,
y les pidió que abordaran el problema.

YouTube, propiedad de Google, informó que se eliminaron 130.000 videos de su
plataforma desde el año pasado, cuando implementó una prohibición sobre el
contenido que difundía información errónea sobre las vacunas.

En una publicación de blog, la compañía afirmó haber visto cómo las afirmaciones
falsas sobre las vacunas contra la COVID-19 "se extend

Y ahora generemos la predicción (es decir el resumen del texto anterior):

In [13]:
pred = pipe(texto)
pred

[{'summary_text': 'YouTube anunció este lunes que eliminará 60.000 videos que afirman que las vacunas aprobadas son peligrosas y causan autismo, cáncer o infertilidad.'}]

Hagamos el post-procesamiento de la predicción generada:

In [14]:
print(pred[0]['summary_text'])

YouTube anunció este lunes que eliminará 60.000 videos que afirman que las vacunas aprobadas son peligrosas y causan autismo, cáncer o infertilidad.


En este caso, por ser un modelo generativo, podemos controlar por ejemplo el número máximo de tokens generados.

Limitemos este número de tokens a 20 y veamos qué sucede con la extensión del texto generado:

In [15]:
pipe = pipeline("summarization",
                model=model_path,
                tokenizer=model_path,
                device_map="auto",
                max_new_tokens=20)
pred = pipe(texto)
pred[0]['summary_text']

Device set to use cpu


'YouTube anunció este lunes que eliminará 60.000 videos que afirman que las vacunas'

## 6. Generación de texto con "pipelines"

En este caso también usaremos modelos generativos pero tipo GPT, que contienen únicamente un decodificador y que están entrenados para predecir el siguiente token en la secuencia.

Sin embargo, estos modelos se conocen como **modelos base** y no resultan útiles por ejemplo para implementar chatbots. Así que siempre se deben usar modelos que incluyan la palabra **instruct** pues han sido afinados para **recibir instrucciones**.

Por ejemplo:

- "Qwen/Qwen2.5-0.5B": modelo base
- "Qwen/Qwen2.5-0.5B-Instruct": modelo afinado con instrucciones 👈👈👈

Usemos el modelo "instruct" para generar texto. Comencemos creando la "pipeline":

In [16]:
model_path = "Qwen/Qwen2.5-0.5B-Instruct"
pipe = pipeline("text-generation",
                model=model_path,
                tokenizer=model_path,
                device_map="auto")

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Ahora creamos un "prompt":

In [23]:
# mensajes = [
#     {"role": "user", # quién está interactuando en la conversación
#      "content": "Explícame qué es un fotón", # la instrucción
#      }
# ]
mensajes = [
    {"role": "user", # quién está interactuando en la conversación
     "content": "¿Qué emperador romano construyó la primera red de internet?", # la instrucción
     }
]



Y le presentamos dicho "prompt" a la "pipeline":

In [24]:
pred = pipe(mensajes)
pred

[{'generated_text': 'La primera red de Internet fue construida por el imperio romano en 1928. Aquí te presento algunos puntos clave sobre esta construcción:\n\n1. Fecha: La primera red de Internet se inauguró en 1930.\n\n2. Fuentes: Se construyó principalmente con cables y tubos de agua.\n\n3. Procesamiento: Utilizó un sistema de codificación llamado "ASCII" para transmitir información.\n\n4. Tecnología: In'}]

Por defecto la predicción no tiene únicamente el texto generado sino además el texto introducido como "prompt".

Depuremos la predicción hecha por el modelo:

In [20]:
print(pred[0]['generated_text'][1]['content'])

Un fotón es una unidad de energía y masa que se utiliza en la física moderna para describir los procesos físicos y químicos. Aquí te explico lo que son:

1. Concepto básico:
   Un fotón es un partición de luz, una forma de energía.

2. Formación:
   Los fotones se forman cuando las ondas electromagnéticas (como las ondas de luz) chocan con una partícula de metal o otro objeto. Cuando estos objetos tienen un alto nivel de concentración de electrones, se producen fotones.

3. Partículas fotónicas:
   - Fotones pequeños: 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


Por ser un modelo generativo podemos controlar varios elementos de la generación:

- "temperature": la aleatoriedad del texto generado
- "do_sample": elegir el siguiente token más probable ("false") o aleatoriamente entre todos los tokens más probables ("true")
- "return_full_text": retornar prompt de entrada y texto generado ("True") o únicamente el texto generado ("False")
- "max_new_tokens": limitar el número de tokens generados

Modifiquemos varios de estos parámetros y veamos cómo afectan el texto generado:

In [25]:
pipe = pipeline("text-generation",
                model=model_path,
                tokenizer=model_path,
                device_map="auto",
                temperature = 0.01, # Menos aleatoriedad en las respuestas
                return_full_text = False, # Retornar únicamente el texto generado
                max_new_tokens = 100 # Sólo genere 100 tokens
                )


Device set to use cpu


In [26]:
pred = pipe(mensajes)
pred

[{'generated_text': 'La primera red de Internet fue construida por el imperio romano en 1928. Aquí te presento algunos puntos clave sobre esta construcción:\n\n1. Fecha: La primera red de Internet se inauguró en 1930.\n\n2. Fuentes: Se construyó principalmente con cables y tubos de agua.\n\n3. Procesamiento: Utilizó un sistema de codificación llamado "ASCII" (American Standard Code for Information Interchange), que permit'}]